In [220]:
import pandas as pd
import numpy as np
import yfinance as yf
import itertools

# Useful tool

In [221]:
def get_stock(ticker):
    # Get data on this ticker
    tickerData = yf.Ticker(ticker)

    # Get the historical prices for this ticker
    stocks = tickerData.history(period='1d', start='2021-1-1', end='2023-1-1')

    stocks.reset_index(inplace=True)
    stocks.Date = pd.to_datetime(stocks.Date).dt.date
    return stocks

In [222]:
stocks = get_stock('MSFT')

In [223]:
def cumulative_reward_long(df, func):
    a = func(df)
    buy_signal = a[0]
    sell_signal = a[1]
    buy_signal = [x for x in buy_signal if x is not np.nan]
    sell_signal = [x for x in sell_signal if x is not np.nan]

    # Assume we buy and sell at the close of the day
    stock_Return = []
    for i in range(0, len(sell_signal)):
        stock_Return.append(sell_signal[i] - buy_signal[i])
    
    return np.sum(stock_Return)

In [224]:
def cumulative_reward_short(df, func):
    a = func(df)
    short_signal = a[0]
    sell_signal = a[1]
    short_signal = [x for x in short_signal if x is not np.nan]
    sell_signal = [x for x in sell_signal if x is not np.nan]

    # Assume we sell and then buy at the close of the day
    stock_Return = []
    for i in range(0, len(sell_signal)):
        stock_Return.append(short_signal[i] - sell_signal[i])
    
    return np.sum(stock_Return)

In [225]:
def split_into_array(s):
    # Remove the leading and trailing square brackets and single quotes
    s = s[1:-1]

    # Split the string into a list of sentences
    integers = s.split(", ")

    return integers

def merge_arrays(series):
    return list(itertools.chain(*series.tolist()))

def prepare_dataset(path, columns):
    df = pandas.read_csv(path)
    
    df['Predictions'] = df['Predictions'].apply(split_into_array).apply(np.array).apply(lambda x: x.astype(int))
    df['Positive'] = df['Positive'].apply(split_into_array).apply(np.array).apply(lambda x: x.astype(float))
    df['Negative'] = df['Negative'].apply(split_into_array).apply(np.array).apply(lambda x: x.astype(float))
    df['Neutral'] = df['Neutral'].apply(split_into_array).apply(np.array).apply(lambda x: x.astype(float))
    df['published'] = pd.to_datetime(df['published']).dt.date
    
    columns_df = columns.copy()
    columns_df.insert(0, 'published')
    
    input = df[columns_df]
    input = input.copy()
    input.rename(columns={'published': 'Date'}, inplace=True)
    
    grouped_df = input.groupby('Date')[columns].agg(merge_arrays)    
    grouped_df = pd.merge(grouped_df, stocks[['Date', 'Close']], on='Date', how='left')

    grouped_df = grouped_df.dropna()
    grouped_df = grouped_df.reset_index(drop=True)

    return grouped_df

# Simple positive negative strategys

In [226]:
def buy_sell_sentiment(signal):
    Buy = [np.nan for i in range(0, len(signal))]
    Sell = [np.nan for i in range(0, len(signal))]
    flag = 0

    for i in range(0, len(signal)):
        if signal['Predictions'][i] == 0:  # Positive sentiment
            if flag != 1:
                Buy[i] = signal['Close'][i]
                flag = 1
        elif signal['Predictions'][i] == 1:  # Negative sentiment
            if flag != 0:
                Sell[i] = signal['Close'][i]
                flag = 0
    return (Buy, Sell)

In [227]:
def short_sell_sentiment(signal):
    Short = [np.nan for i in range(0, len(signal))]
    Sell = [np.nan for i in range(0, len(signal))]
    flag = 0

    for i in range(0, len(signal)):
        if signal['Predictions'][i] == 1:  # Negative sentiment
            if flag != 1:
                Short[i] = signal['Close'][i]
                flag = 1
        elif signal['Predictions'][i] == 0:  # Positive sentiment
            if flag != 0:
                Sell[i] = signal['Close'][i]
                flag = 0
    return (Short, Sell)

Simple first implementation we look at the sentiment of the news and if it is positive we buy and if it is negative we sell. We will use the sentiment of the news from the previous day to make our decision, by mean of the sentences in a same day.

In [228]:
simple_news_dataset = prepare_dataset("/Users/tony/Desktop/Antoine/Semestre/9.Financial ML/ProjectFinancialNews/data/msft_roberta_output.csv", ['Predictions'])

In [229]:
simple_news_dataset

,Date,Predictions,Close
0,2021-01-04,"[2, 0, 2, 2, 2, 0, 2]",211.996613
1,2021-01-05,"[2, 2, 1, 2, 2, 0, 2, 2, 0, 2, 0, 2, 2, 2, 0, ...",212.201096
2,2021-01-06,"[0, 0, 2, 2, 2, 0, 0, 2, 0, 2, 2, 2, 2]",206.698868
3,2021-01-07,"[2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, ...",212.580902
4,2021-01-08,"[2, 2, 2, 2, 2, 2, 2, 2]",213.876160
...,...,...,...
498,2022-12-23,"[1, 2, 2, 2, 1, 0, 0, 2, 1, 0, 0, 0, 2, 2, 1, ...",236.631805
499,2022-12-27,"[2, 2, 0, 0, 2, 0]",234.877365
500,2022-12-28,"[0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 1, 1, 2, 2, 2, ...",232.468719
501,2022-12-29,"[2, 0, 2, 2, 2, 0, 1, 2, 2, 2, 2, 1, 2, 1, 1, ...",238.891769


In [230]:
from collections import Counter

def take_max_count_prediction(lst):
    counts = Counter(lst)
    max_count = max(counts.values())
    modes = [k for k, v in counts.items() if v == max_count]
    
    if len(modes) == 1:
        return modes[0]
    elif len(modes) == 3:
        return 2
    elif 0 in modes and 1 in modes:
        return 2
    elif 0 in modes and 2 in modes:
        return 0
    elif 1 in modes and 2 in modes:
        return 1
    else:
        return 2

simple_news_dataset['Predictions'] = simple_news_dataset['Predictions'].apply(take_max_count_prediction)

In [231]:
simple_news_dataset.head()

,Date,Predictions,Close
0,2021-01-04,2,211.996613
1,2021-01-05,2,212.201096
2,2021-01-06,2,206.698868
3,2021-01-07,2,212.580902
4,2021-01-08,2,213.876160


In [232]:
print("Long Cumulative Return : ", cumulative_reward_long(simple_news_dataset, buy_sell_sentiment))
print("Short Cumulative Return : ", cumulative_reward_short(simple_news_dataset, short_sell_sentiment))

Long Cumulative Return :  52.32215881347656
Short Cumulative Return :  16.449050903320312


Now we take the maximum of the different sentences per day between positive and Negative

In [233]:
max_sum_score = prepare_dataset("/Users/tony/Desktop/Antoine/Semestre/9.Financial ML/ProjectFinancialNews/data/msft_roberta_output.csv", ['Positive', 'Negative', 'Neutral'])
max_sum_score.head()

,Date,Positive,Negative,Neutral,Close
0,2021-01-04,"[5.761160355177708e-05, 0.7729113101959229, 8....","[8.459905802737921e-05, 0.0005494784563779831,...","[0.9998577833175659, 0.2265392392873764, 0.999...",211.996613
1,2021-01-05,"[6.80618395563215e-05, 7.606166764162481e-05, ...","[6.462157034548e-05, 6.527246296172962e-05, 0....","[0.99986732006073, 0.9998586177825928, 0.00241...",212.201096
2,2021-01-06,"[0.9927652478218079, 0.9984225034713745, 5.471...","[0.00025216813082806766, 0.0001982791873160749...","[0.006982621271163225, 0.0013791749952360988, ...",206.698868
3,2021-01-07,"[7.9099896538537e-05, 5.7219538575736806e-05, ...","[0.0005839336663484573, 0.00021778183872811496...","[0.9993368983268738, 0.9997250437736511, 0.999...",212.580902
4,2021-01-08,"[4.7886129323160276e-05, 4.737492781714536e-05...","[6.148114334791899e-05, 6.220681098056957e-05,...","[0.9998905658721924, 0.9998904466629028, 0.999...",213.876160


In [234]:
max_sum_score[['Positive', 'Negative', 'Neutral']] = max_sum_score[['Positive', 'Negative', 'Neutral']].applymap(sum)
max_sum_score.head()

,Date,Positive,Negative,Neutral,Close
0,2021-01-04,1.723691,0.001351,5.274958,211.996613
1,2021-01-05,7.853685,3.949336,17.196979,212.201096
2,2021-01-06,4.950893,0.122638,7.926469,206.698868
3,2021-01-07,0.013350,3.293995,18.692655,212.580902
4,2021-01-08,0.000524,0.000791,7.998685,213.876160


In [235]:
# Python
def take_max_count_prediction(row):
    values = {'Positive': row['Positive'], 'Negative': row['Negative'], 'Neutral': row['Neutral']}
    max_key = max(values, key=values.get)
    
    if max_key == 'Positive':
        return 0
    elif max_key == 'Negative':
        return 1
    else:
        return 2

max_sum_score['Predictions'] = max_sum_score.apply(take_max_count_prediction, axis=1)

In [236]:
max_sum_score.head()

,Date,Positive,Negative,Neutral,Close,Predictions
0,2021-01-04,1.723691,0.001351,5.274958,211.996613,2
1,2021-01-05,7.853685,3.949336,17.196979,212.201096,2
2,2021-01-06,4.950893,0.122638,7.926469,206.698868,2
3,2021-01-07,0.013350,3.293995,18.692655,212.580902,2
4,2021-01-08,0.000524,0.000791,7.998685,213.876160,2


In [237]:
print("Long Cumulative Return : ", cumulative_reward_long(max_sum_score, buy_sell_sentiment))
print("Short Cumulative Return : ", cumulative_reward_short(max_sum_score, short_sell_sentiment))

Long Cumulative Return :  28.418701171875
Short Cumulative Return :  -3.0136566162109375


We perform less with this technique

We take some threshold only the value greater than this into account, with mean of the day

In [238]:
threshold_input = prepare_dataset("/Users/tony/Desktop/Antoine/Semestre/9.Financial ML/ProjectFinancialNews/data/msft_roberta_output.csv", ['Positive', 'Negative', 'Neutral'])
THRESHOLD = 0.5

In [239]:
def replace_below_threshold(arr, threshold):
    return [x for x in arr if x >= threshold]

columns = ['Positive', 'Negative', 'Neutral']

for col in columns:
    threshold_input[col] = threshold_input[col].apply(lambda arr: replace_below_threshold(arr, threshold=THRESHOLD))

In [240]:
threshold_input.head()

,Date,Positive,Negative,Neutral,Close
0,2021-01-04,"[0.7729113101959229, 0.9503400325775146]",[],"[0.9998577833175659, 0.9998621940612793, 0.999...",211.996613
1,2021-01-05,"[0.9276857972145081, 0.9853912591934204, 0.997...","[0.9968718886375427, 0.9968718886375427, 0.996...","[0.99986732006073, 0.9998586177825928, 0.99975...",212.201096
2,2021-01-06,"[0.9927652478218079, 0.9984225034713745, 0.992...",[],"[0.9998841285705566, 0.878739595413208, 0.9998...",206.698868
3,2021-01-07,[],"[0.9986888766288757, 0.9969198703765869, 0.998...","[0.9993368983268738, 0.9997250437736511, 0.999...",212.580902
4,2021-01-08,[],[],"[0.9998905658721924, 0.9998904466629028, 0.999...",213.876160


In [241]:
threshold_input[['Positive', 'Negative', 'Neutral']] = threshold_input[['Positive', 'Negative', 'Neutral']].applymap(len)

In [242]:
threshold_input.head()

,Date,Positive,Negative,Neutral,Close
0,2021-01-04,2,0,5,211.996613
1,2021-01-05,8,3,17,212.201096
2,2021-01-06,5,0,8,206.698868
3,2021-01-07,0,3,19,212.580902
4,2021-01-08,0,0,8,213.876160


In [243]:
threshold_input['Predictions'] = threshold_input.apply(take_max_count_prediction, axis=1)

In [244]:
threshold_input.head()

,Date,Positive,Negative,Neutral,Close,Predictions
0,2021-01-04,2,0,5,211.996613,2
1,2021-01-05,8,3,17,212.201096,2
2,2021-01-06,5,0,8,206.698868,2
3,2021-01-07,0,3,19,212.580902,2
4,2021-01-08,0,0,8,213.876160,2


In [245]:
print("Long Cumulative Return : ", cumulative_reward_long(threshold_input, buy_sell_sentiment))
print("Short Cumulative Return : ", cumulative_reward_short(threshold_input, short_sell_sentiment))

Long Cumulative Return :  45.48687744140625
Short Cumulative Return :  9.61376953125


In [246]:
def threshold_test():
    THRESHOLD_ARRAY = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.85, 0.9, 0.95, 0.99]
    for THRESHOLD in THRESHOLD_ARRAY:
        threshold_input = prepare_dataset("/Users/tony/Desktop/Antoine/Semestre/9.Financial ML/ProjectFinancialNews/data/msft_roberta_output.csv", ['Positive', 'Negative', 'Neutral'])
        
        columns = ['Positive', 'Negative', 'Neutral']
        for col in columns:
            threshold_input[col] = threshold_input[col].apply(lambda arr: replace_below_threshold(arr, threshold=THRESHOLD))        
            
        threshold_input[['Positive', 'Negative', 'Neutral']] = threshold_input[['Positive', 'Negative', 'Neutral']].applymap(len)
        
        threshold_input['Predictions'] = threshold_input.apply(take_max_count_prediction, axis=1)
        
        print("Long Cumulative Return : ", cumulative_reward_long(threshold_input, buy_sell_sentiment))
        print("Short Cumulative Return : ", cumulative_reward_short(threshold_input, short_sell_sentiment))

In [247]:
threshold_test()

Long Cumulative Return :  48.290130615234375
Short Cumulative Return :  12.417022705078125
Long Cumulative Return :  47.42265319824219
Short Cumulative Return :  11.549545288085938
Long Cumulative Return :  47.42265319824219
Short Cumulative Return :  11.549545288085938
Long Cumulative Return :  41.087860107421875
Short Cumulative Return :  5.214752197265625
Long Cumulative Return :  45.48687744140625
Short Cumulative Return :  9.61376953125
Long Cumulative Return :  45.48687744140625
Short Cumulative Return :  9.61376953125
Long Cumulative Return :  51.82167053222656
Short Cumulative Return :  15.948562622070312
Long Cumulative Return :  47.42265319824219
Short Cumulative Return :  11.549545288085938
Long Cumulative Return :  32.732513427734375
Short Cumulative Return :  -3.140594482421875
Long Cumulative Return :  32.732513427734375
Short Cumulative Return :  -3.140594482421875
Long Cumulative Return :  37.13153076171875
Short Cumulative Return :  1.2584228515625
Long Cumulative Retu

Weird, better is the filter the worst is the result

# Momemntum X News trading strategy

We are going to use the better we saw at this stage the Threshold 0.7

## With MACD

## AND need both signals to be positive

In [248]:
def buy_sell_momentum_sentiment(signal):
    Buy = [np.nan for i in range(0, len(signal))]
    Sell = [np.nan for i in range(0, len(signal))]
    flag = 0

    for i in range(0, len(signal)):
        if signal['MACD'][i] > signal['Signal Line'][i] and signal['Predictions'][i] == 0:  # Positive momentum and sentiment
            if flag != 1:
                Buy[i] = signal['Close'][i]
                flag = 1
        elif signal['MACD'][i] < signal['Signal Line'][i] and signal['Predictions'][i] == 1:  # Negative momentum and sentiment
            if flag != 0:
                Sell[i] = signal['Close'][i]
                flag = 0

    return (Buy, Sell)

In [249]:
def short_sell_momentum_sentiment(signal):
    Short = [np.nan for i in range(0, len(signal))]
    Sell = [np.nan for i in range(0, len(signal))]
    flag = 0

    for i in range(0, len(signal)):
        if signal['MACD'][i] < signal['Signal Line'][i] and signal['Predictions'][i] == 1:  # Negative momentum and sentiment
            if flag != 1:
                Short[i] = signal['Close'][i]
                flag = 1
        elif signal['MACD'][i] > signal['Signal Line'][i] and signal['Predictions'][i] == 0:  # Positive momentum and sentiment
            if flag != 0:
                Sell[i] = signal['Close'][i]
                flag = 0

    return (Short, Sell)

In [250]:
stocks = get_stock('MSFT')

In [251]:
# Calculate Short Term Exponential Moving Average
ShortEMA = stocks.Close.ewm(span=12, adjust=False).mean() 
# Calculate Long Term Exponential Moving Average
LongEMA = stocks.Close.ewm(span=26, adjust=False).mean() 
# Calculate MACD line
MACD = ShortEMA - LongEMA
# Calculate Signal line
signal = MACD.ewm(span=9, adjust=False).mean()

In [252]:
stocks['MACD'] = MACD
stocks['Signal Line'] = signal

In [253]:
THRESHOLD = 0.7

MACD_news_input = prepare_dataset("/Users/tony/Desktop/Antoine/Semestre/9.Financial ML/ProjectFinancialNews/data/msft_roberta_output.csv", ['Positive', 'Negative', 'Neutral'])
MACD_news_input = pd.merge(MACD_news_input, stocks[['Date', 'MACD', 'Signal Line']], on='Date', how='left')

columns = ['Positive', 'Negative', 'Neutral']

for col in columns:
    MACD_news_input[col] = MACD_news_input[col].apply(lambda arr: replace_below_threshold(arr, threshold=THRESHOLD))        
    
MACD_news_input[['Positive', 'Negative', 'Neutral']] = MACD_news_input[['Positive', 'Negative', 'Neutral']].applymap(len)

MACD_news_input['Predictions'] = MACD_news_input.apply(take_max_count_prediction, axis=1)

print("Threshold : ", THRESHOLD, " Long Cumulative Return : ", cumulative_reward_long(MACD_news_input, buy_sell_momentum_sentiment))
print("Threshold : ", THRESHOLD, " Short Cumulative Return : ", cumulative_reward_short(MACD_news_input, short_sell_momentum_sentiment))


Threshold :  0.7  Long Cumulative Return :  20.96783447265625
Threshold :  0.7  Short Cumulative Return :  -14.9052734375


## OR need one of the signal to be positive

In [254]:
def buy_sell_momentum_sentiment(signal):
    Buy = [np.nan for i in range(0, len(signal))]
    Sell = [np.nan for i in range(0, len(signal))]
    flag = 0

    for i in range(0, len(signal)):
        if signal['MACD'][i] > signal['Signal Line'][i] or signal['Predictions'][i] == 0:  # Positive momentum and sentiment
            if flag != 1:
                Buy[i] = signal['Close'][i]
                flag = 1
        elif signal['MACD'][i] < signal['Signal Line'][i] or signal['Predictions'][i] == 1:  # Negative momentum and sentiment
            if flag != 0:
                Sell[i] = signal['Close'][i]
                flag = 0

    return (Buy, Sell)

In [255]:
def short_sell_momentum_sentiment(signal):
    Short = [np.nan for i in range(0, len(signal))]
    Sell = [np.nan for i in range(0, len(signal))]
    flag = 0

    for i in range(0, len(signal)):
        if signal['MACD'][i] < signal['Signal Line'][i] or signal['Predictions'][i] == 1:  # Negative momentum and sentiment
            if flag != 1:
                Short[i] = signal['Close'][i]
                flag = 1
        elif signal['MACD'][i] > signal['Signal Line'][i] or signal['Predictions'][i] == 0:  # Positive momentum and sentiment
            if flag != 0:
                Sell[i] = signal['Close'][i]
                flag = 0

    return (Short, Sell)

In [256]:
stocks = get_stock('MSFT')

In [257]:
# Calculate Short Term Exponential Moving Average
ShortEMA = stocks.Close.ewm(span=12, adjust=False).mean() 
# Calculate Long Term Exponential Moving Average
LongEMA = stocks.Close.ewm(span=26, adjust=False).mean() 
# Calculate MACD line
MACD = ShortEMA - LongEMA
# Calculate Signal line
signal = MACD.ewm(span=9, adjust=False).mean()

In [258]:
stocks['MACD'] = MACD
stocks['Signal Line'] = signal

In [259]:
THRESHOLD = 0.7

MACD_news_input = prepare_dataset("/Users/tony/Desktop/Antoine/Semestre/9.Financial ML/ProjectFinancialNews/data/msft_roberta_output.csv", ['Positive', 'Negative', 'Neutral'])
MACD_news_input = pd.merge(MACD_news_input, stocks[['Date', 'MACD', 'Signal Line']], on='Date', how='left')

columns = ['Positive', 'Negative', 'Neutral']

for col in columns:
    MACD_news_input[col] = MACD_news_input[col].apply(lambda arr: replace_below_threshold(arr, threshold=THRESHOLD))        
    
MACD_news_input[['Positive', 'Negative', 'Neutral']] = MACD_news_input[['Positive', 'Negative', 'Neutral']].applymap(len)

MACD_news_input['Predictions'] = MACD_news_input.apply(take_max_count_prediction, axis=1)

print("Threshold : ", THRESHOLD, " Long Cumulative Return : ", cumulative_reward_long(MACD_news_input, buy_sell_momentum_sentiment))
print("Threshold : ", THRESHOLD, " Short Cumulative Return : ", cumulative_reward_short(MACD_news_input, short_sell_momentum_sentiment))

Threshold :  0.7  Long Cumulative Return :  10.322006225585938
Threshold :  0.7  Short Cumulative Return :  -38.47743225097656


## With RSI

## AND need both signals to be positive

In [260]:
def buy_sell_RSI_sentiment(signal, upper=70, lower=30):
    Buy = [np.nan for i in range(0, len(signal))]
    Sell = [np.nan for i in range(0, len(signal))]
    flag = 0

    for i in range(0, len(signal)):
        if signal['RSI'][i] < lower and signal['Predictions'][i] == 0:
            if flag != 1:
                Buy[i] = signal['Close'][i]
                flag = 1
        elif signal['RSI'][i] > upper and signal['Predictions'][i] == 1:
            if flag != 0:
                Sell[i] = signal['Close'][i]
                flag = 0

    return (Buy, Sell)

In [261]:
def short_sell_RSI_sentiment(signal, upper=70, lower=30):
    Short = [np.nan for i in range(0, len(signal))]
    Sell = [np.nan for i in range(0, len(signal))]
    flag = 0

    for i in range(0, len(signal)):
        if signal['RSI'][i] > upper and signal['Predictions'][i] == 1:
            if flag != 1:
                Short[i] = signal['Close'][i]
                flag = 1
        elif signal['RSI'][i] < lower and signal['Predictions'][i] == 0:
            if flag != 0:
                Sell[i] = signal['Close'][i]
                flag = 0

    return (Short, Sell)

In [262]:
def calculate_RSI(data, time_window):
    diff = data.diff(1).dropna()        # diff in one field(one day)

    #this preservers dimensions off diff values
    up_chg = 0 * diff
    down_chg = 0 * diff
    
    # up change is equal to the positive difference, otherwise equal to zero
    up_chg[diff > 0] = diff[ diff>0 ]
    
    # down change is equal to negative deifference, otherwise equal to zero
    down_chg[diff < 0] = diff[ diff < 0 ]

    # we set com=time_window-1 so we get decay alpha=1/time_window
    up_chg_avg   = up_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi

stocks = get_stock('MSFT')

stocks['RSI'] = calculate_RSI(stocks['Close'], 14)

In [263]:
THRESHOLD = 0.7

RSI_news_input = prepare_dataset("/Users/tony/Desktop/Antoine/Semestre/9.Financial ML/ProjectFinancialNews/data/msft_roberta_output.csv", ['Positive', 'Negative', 'Neutral'])
RSI_news_input = pd.merge(RSI_news_input, stocks[['Date', 'RSI']], on='Date', how='left')

columns = ['Positive', 'Negative', 'Neutral']

for col in columns:
    RSI_news_input[col] = RSI_news_input[col].apply(lambda arr: replace_below_threshold(arr, threshold=THRESHOLD))        
    
RSI_news_input[['Positive', 'Negative', 'Neutral']] = RSI_news_input[['Positive', 'Negative', 'Neutral']].applymap(len)

RSI_news_input['Predictions'] = RSI_news_input.apply(take_max_count_prediction, axis=1)

print("Threshold : ", THRESHOLD, " Long Cumulative Return : ", cumulative_reward_long(RSI_news_input, buy_sell_RSI_sentiment))
print("Threshold : ", THRESHOLD, " Short Cumulative Return : ", cumulative_reward_short(RSI_news_input, short_sell_RSI_sentiment))

Threshold :  0.7  Long Cumulative Return :  0.0
Threshold :  0.7  Short Cumulative Return :  -44.938873291015625


## OR need one of the signal to be positive

In [264]:
def buy_sell_RSI_sentiment(signal, upper=70, lower=30):
    Buy = [np.nan for i in range(0, len(signal))]
    Sell = [np.nan for i in range(0, len(signal))]
    flag = -1

    for i in range(0, len(signal)):
        if signal['RSI'][i] < lower or signal['Predictions'][i] == 0:
            if flag != 1:
                Buy[i] = signal['Close'][i]
                flag = 1
        elif signal['RSI'][i] > upper or signal['Predictions'][i] == 1:
            if flag != 0:
                Sell[i] = signal['Close'][i]
                flag = 0

    return (Buy, Sell)

In [265]:
def short_sell_RSI_sentiment(signal, upper=70, lower=30):
    Short = [np.nan for i in range(0, len(signal))]
    Sell = [np.nan for i in range(0, len(signal))]
    flag = 0

    for i in range(0, len(signal)):
        if signal['RSI'][i] > upper or signal['Predictions'][i] == 1:
            if flag != 1:
                Short[i] = signal['Close'][i]
                flag = 1
        elif signal['RSI'][i] < lower or signal['Predictions'][i] == 0:
            if flag != 0:
                Sell[i] = signal['Close'][i]
                flag = 0

    return (Short, Sell)

In [266]:
THRESHOLD = 0.7

RSI_news_input = prepare_dataset("/Users/tony/Desktop/Antoine/Semestre/9.Financial ML/ProjectFinancialNews/data/msft_roberta_output.csv", ['Positive', 'Negative', 'Neutral'])
RSI_news_input = pd.merge(RSI_news_input, stocks[['Date', 'RSI']], on='Date', how='left')

columns = ['Positive', 'Negative', 'Neutral']

for col in columns:
    RSI_news_input[col] = RSI_news_input[col].apply(lambda arr: replace_below_threshold(arr, threshold=THRESHOLD))        
    
RSI_news_input[['Positive', 'Negative', 'Neutral']] = RSI_news_input[['Positive', 'Negative', 'Neutral']].applymap(len)

RSI_news_input['Predictions'] = RSI_news_input.apply(take_max_count_prediction, axis=1)

print("Threshold : ", THRESHOLD, " Long Cumulative Return : ", cumulative_reward_long(RSI_news_input, buy_sell_RSI_sentiment))
print("Threshold : ", THRESHOLD, " Short Cumulative Return : ", cumulative_reward_short(RSI_news_input, short_sell_RSI_sentiment))

Threshold :  0.7  Long Cumulative Return :  21.994903564453125
Threshold :  0.7  Short Cumulative Return :  -24.432327270507812
